In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.tsv", sep="\t", index_col=0) # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)   # 評価用データ
gender_submission = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None) # 応募用サンプルファイル

In [2]:
train.describe()

,survived,pclass,age,sibsp,parch,fare
count,445.000000,445.000000,360.000000,445.000000,445.000000,445.000000
mean,0.402247,2.296629,29.211583,0.546067,0.431461,33.959971
std,0.490903,0.834024,14.154300,1.195247,0.850489,52.079492
min,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,15.000000
75%,1.000000,3.000000,37.250000,1.000000,1.000000,31.387500
max,1.000000,3.000000,80.000000,8.000000,5.000000,512.329200


In [3]:
test.describe()

,pclass,age,sibsp,parch,fare
count,446.000000,354.000000,446.000000,446.000000,446.000000
mean,2.320628,30.194915,0.500000,0.331839,30.452381
std,0.838873,14.899140,1.002805,0.756823,47.186192
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,28.250000,0.000000,0.000000,13.500000
75%,3.000000,39.000000,1.000000,0.000000,30.000000
max,3.000000,74.000000,8.000000,6.000000,512.329200


In [4]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
7,0,3,male,2.0,3,1,21.0750,S
9,1,2,female,14.0,1,0,30.0708,C
11,1,1,female,58.0,0,0,26.5500,S


In [5]:
test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
5,3,male,NaN,0,0,8.4583,Q
6,1,male,54.0,0,0,51.8625,S


In [6]:
gender_submission.head()

,1
0,
0,0
1,1
2,0
5,1
6,1


## 1.データ結合

In [7]:
data = pd.concat([train, test], sort=False)

In [8]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1.0,1,female,35.0,1,0,53.1000,S
4,0.0,3,male,35.0,0,0,8.0500,S
7,0.0,3,male,2.0,3,1,21.0750,S
9,1.0,2,female,14.0,1,0,30.0708,C
11,1.0,1,female,58.0,0,0,26.5500,S


In [9]:
print(len(train), len(test), len(data))

445 446 891


### 2.Sex

In [10]:
data['sex'].replace(['male', 'female'], [0, 1], inplace=True)

### 3. Emarked

In [11]:
data['embarked'].fillna(('S'), inplace=True)
data['embarked'] = data['embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

### Fare

In [12]:
data['age'].fillna(data['age'].median(), inplace=True)

In [13]:
### 6.FamilySizeの追加¶

In [14]:
data['familySize'] = data['parch'] + data['sibsp'] + 1
train['familySize'] = data['familySize'][:len(train)]
test['familySize'] = data['familySize'][len(train):]

### 7.Aloneの追加¶

In [15]:
data['IsAlone'] = 0
data.loc[data['familySize']==1, 'IsAlone'] = 1

train['IsAlone'] = data['IsAlone'][:len(train)]
test['IsAlone']=data['IsAlone'][len(train):]

In [16]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,familySize,IsAlone
id,,,,,,,,,,
3,1.0,1,1,35.0,1,0,53.1000,0,2,0
4,0.0,3,0,35.0,0,0,8.0500,0,1,1
7,0.0,3,0,2.0,3,1,21.0750,0,5,0
9,1.0,2,1,14.0,1,0,30.0708,1,2,0
11,1.0,1,1,58.0,0,0,26.5500,0,1,1


### 必要ない項目の削除

In [17]:
delete_columns = [ 'sibsp', 'parch']
data.drop(delete_columns, axis=1, inplace=True)

### データの分離

In [18]:
train = data[:len(train)]
test = data[len(train):]

In [19]:
y_train = train['survived']
X_train = train.drop('survived', axis=1)
X_test = test.drop('survived', axis=1)

In [20]:
X_train.head()

,pclass,sex,age,fare,embarked,familySize,IsAlone
id,,,,,,,
3,1,1,35.0,53.1000,0,2,0
4,3,0,35.0,8.0500,0,1,1
7,3,0,2.0,21.0750,0,5,0
9,2,1,14.0,30.0708,1,2,0
11,1,1,58.0,26.5500,0,1,1


In [21]:
y_train.head()

id
3     1.0
4     0.0
7     0.0
9     1.0
11    1.0
Name: survived, dtype: float64

### LightGBM 事前準備

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                                 random_state=0, stratify=y_train)


### カテゴリ変数をリスト形式で宣言

In [23]:
categorical_features = ['embarked', 'pclass', 'sex']

In [24]:
import lightgbm as lgb

import optuna
from sklearn.metrics import log_loss


def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score


In [25]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2021-06-11 14:35:07,399] A new study created in memory with name: no-name-0bc6e25d-4a89-4397-9152-875b0e3cd8b1
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-06-11 14:35:07,475] Trial 0 finished with value: 0.49153935535093457 and parameters: {'max_bin': 390, 'num_leaves': 101}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('U

[LightGBM] [Info] Number of positive: 125, number of negative: 186
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 311, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401929 -> initscore=-0.397433
[LightGBM] [Info] Start training from score -0.397433
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

[I 2021-06-11 14:35:07,612] Trial 2 finished with value: 0.49153935535093457 and parameters: {'max_bin': 359, 'num_leaves': 94}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-06-11 14:35:07,680] Trial 3 finished with value: 0.49153935535093457 and parameters: {'max_bin': 362, 'num_leaves': 118}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserW

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	training's binary_logloss: 0.317909	valid_1's binary_logloss: 0.49646
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[52]	training's binary_logloss: 0.332408	valid_1's binary_logloss: 0.491539
[LightGBM] [Info] Number of positive: 125, number of negative: 186
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 311, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401929 -> initscore=-0.397433
[LightGBM] [Inf

C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-06-11 14:35:07,815] Trial 5 finished with value: 0.49153935535093457 and parameters: {'max_bin': 449, 'num_leaves': 83}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarni

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[40]	training's binary_logloss: 0.362373	valid_1's binary_logloss: 0.495545
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-06-11 14:35:09,898] Trial 35 finished with value: 0.49153935535093457 and parameters: {'max_bin': 495, 'num_leaves': 77}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarn

No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	training's binary_logloss: 0.522822	valid_1's binary_logloss: 0.562805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	training's binary_logloss: 0.449477	vali

C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-06-11 14:35:10,102] Trial 38 finished with value: 0.49153935535093457 and parameters: {'max_bin': 324, 'num_leaves': 43}. Best is trial 0 with value: 0.49153935535093457.
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarn

[LightGBM] [Info] Number of positive: 125, number of negative: 186
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 311, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401929 -> initscore=-0.397433
[LightGBM] [Info] Start training from score -0.397433
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

In [26]:
study.best_params

{'max_bin': 390, 'num_leaves': 101}

In [27]:
params = {
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': 0.05,
    'num_leaves': study.best_params['num_leaves']
}

lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                         categorical_feature=categorical_features)

model = lgb.train(params, lgb_train,
                               valid_sets=[lgb_train, lgb_eval],
                               verbose_eval=10,
                               num_boost_round=1000,
                               early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\takai\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Number of positive: 125, number of negative: 186
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000091 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 311, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401929 -> initscore=-0.397433
[LightGBM] [Info] Start training from score -0.397433
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

In [28]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 1, 1, 0, 0, 1, 1, 0, 1, 1])

In [29]:
y_pred = list(y_pred)
y_pred[:10]

[0, 1, 1, 0, 0, 1, 1, 0, 1, 1]

### 提出

In [30]:
gender_submission[1] = y_pred
gender_submission.to_csv('submit.tsv', header=None, sep='\t')

In [31]:
gender_submission.head()

,1
0,
0,0
1,1
2,1
5,0
6,0
